In [1]:
#Nathan Li - Final Titanic Survival Model (Hybrid)

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score #used instead of MAE because the Titanic problem is a classification type, not regressiosn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
#Process the names of people to extract their titles
Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Dr": "Officer",
    "Rev": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir": "Royalty",
    "the Countess": "Royalty",
    "Lady": "Royalty",
    "Mlle": "Miss",
    "Miss": "Miss",
    "Mme": "Mrs",
    "Ms": "Mrs",
    "Mrs": "Mrs",
    "Mr": "Mr",
    "Master": "Master"
}

def get_titles(dataset):
    #extract title from each name
    dataset['Title'] = dataset['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
    
    #map each title
    dataset['Title'] = dataset.Title.map(Title_Dictionary)

get_titles(train_data)
get_titles(test_data)

In [7]:
#Fill missing Age values based on Pclass, Title, and Sex
grouped_train_data = train_data.groupby(["Pclass", "Title", "Sex"])
grouped_median_train_data = grouped_train_data.median()
grouped_median_train_data = grouped_median_train_data.reset_index()[["Pclass", "Title", "Sex", "Age"]]

grouped_test_data = test_data.groupby(["Pclass", "Title", "Sex"])
grouped_median_test_data = grouped_test_data.median()
grouped_median_test_data = grouped_median_test_data.reset_index()[["Pclass", "Title", "Sex", "Age"]]

def fill_age(row, grouped_dataset):
    condition = (
        (grouped_dataset["Pclass"] == row["Pclass"]) &
        (grouped_dataset["Title"] == row["Title"]) &
        (grouped_dataset["Sex"] == row["Sex"])
    )
    if np.isnan(grouped_dataset[condition]['Age'].values[0]):
        condition = (
            (grouped_dataset["Pclass"] == row["Pclass"]) &
            (grouped_dataset["Sex"] == row["Sex"])
        )
    
    return grouped_dataset[condition]['Age'].values[0]

train_data["Age"] = train_data.apply(lambda row: fill_age(row, grouped_median_train_data) if np.isnan(row['Age']) else row['Age'], axis=1)
test_data["Age"] = test_data.apply(lambda row: fill_age(row, grouped_median_test_data) if np.isnan(row['Age']) else row['Age'], axis=1)

In [8]:
#Fill missing Embarked values with mode, S
train_data["Embarked"]=train_data["Embarked"].fillna("S")
test_data["Embarked"]=test_data["Embarked"].fillna("S")

#Filling empty Fare value with the median Fare
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].median())

In [9]:
#Dropping the features that will not be used
train_data = train_data.drop(["PassengerId", "Name", "Ticket", "Cabin", "Title"], axis = 1)
test_data = test_data.drop(["Name", "Ticket", "Cabin", "Title"], axis = 1)

In [10]:
le = LabelEncoder()

#Turning the categorical Sex values to numerical
train_data["Sex"] = le.fit_transform(train_data["Sex"])
test_data["Sex"] = le.fit_transform(test_data["Sex"])

#Turning the categorical Embarked values to numerical with a method similar to hot-end encoding
train_data = pd.get_dummies(train_data, columns = ["Embarked"])
test_data = pd.get_dummies(test_data, columns = ["Embarked"])
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


In [11]:
#Using train test split
y = train_data["Survived"]
X = train_data.drop("Survived", axis = 1)
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [12]:
#I didn't bother changing the Gradient Boosting Classifier parameters, but I may be able to squeeze out more accuracy by messing around with it later
model = GradientBoostingClassifier(random_state=1)
model.fit(train_X, train_y)
predictions = model.predict(val_X)

print('The accuracy of the Gradient Boosting Classifier is',round(accuracy_score(val_y,predictions)*100,2))

The accuracy of the Gradient Boosting Classifier is 78.77


In [13]:
#Test for submission
X_test = test_data.drop("PassengerId", axis = 1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
